In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from aac_crud import AnimalShelterCRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = 'aacuser'
password = 'SNHU1234'
HOST = 'nv-desktop-services.apporto.com'
PORT = 31813
DB = 'AAC'
COL = 'animals'
shelter = AnimalShelterCRUD(username, password, HOST, PORT, DB, COL)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
#########################
# Dashboard Layout / View
#########################
image_filename = 'Grazioso_Salvare_Logo.png'
app = JupyterDash('Mike Example')
app.layout = html.Div([

    html.Center(html.B(html.H1('CS-340 Dashboard Final Project'))),
    html.Hr(),

    # Logo and Header
    html.Div(className='row', style={'display': 'flex', 'margin': '10px'}, children=[
        html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height': '5%', 'width': '5%'}),
        html.Center(html.B(html.H1('Grazioso Salvare Dashboard - CS340 - Michael Clerico')))
    ]),

    # Dropdown Filter
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Rescue', 'value': 'Disaster'}
            ],
            value='All',
            placeholder="Filter by Rescue Type"
        )
    ], style={'width': '40%', 'padding': '10px'}),

    html.Hr(),

    # Interactive Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),

    html.Br(),
    html.Hr(),

    # Graph and Map side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])



selected_rows=[0],


#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Water':
        query = {
            "$and": [
                {"$or": [
                    {'breed': {"$regex": 'Labrador Retriever Mix', "$options": "i"}},
                    {'breed': {"$regex": 'Chesapeake', "$options": "i"}},
                    {'breed': {"$regex": 'Newfoundland', "$options": "i"}}
                ]},
                {'sex_upon_outcome': 'Intact Female'},
                {'age_upon_outcome_in_weeks': {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'Mountain':
        query = {
            "$and": [
                {"$or": [
                    {'breed': {"$regex": 'German Shepherd', "$options": "i"}},
                    {'breed': {"$regex": 'Alaskan Malamute', "$options": "i"}},
                    {'breed': {"$regex": 'Old English Sheepdog', "$options": "i"}},
                    {'breed': {"$regex": 'Siberian Husky', "$options": "i"}},
                    {'breed': {"$regex": 'Rottweiler', "$options": "i"}}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'Disaster':
        query = {
            "$and": [
                {"$or": [
                    {'breed': {"$regex": 'Doberman Pinscher', "$options": "i"}},
                    {'breed': {"$regex": 'German Shepherd', "$options": "i"}},
                    {'breed': {"$regex": 'Golden Retriever', "$options": "i"}},
                    {'breed': {"$regex": 'Bloodhound', "$options": "i"}},
                    {'breed': {"$regex": 'Rottweiler', "$options": "i"}}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {"$gte": 20, "$lte": 300}}
            ]
        }
    else:
        query = {'animal_type': 'Dog'}

    # Query MongoDB
    records = shelter.read(query)
    df = pd.DataFrame.from_records(records)

    # Remove MongoDB _id field if present
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available to plot.")]

    df = pd.DataFrame.from_records(viewData)

    if 'breed' not in df.columns or df['breed'].dropna().empty:
        return [html.P("No breed data available.")]

    return [
        dcc.Graph(
            figure=px.pie(df, names='breed', title='Preferred Animals')
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    return[
        dl.Map(style={'width': '1000px', 'height': '500px'},
        center=[30.75, -97, 48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
            children=[
            dl.Tooltip(dff.iloc[row, 4]),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
            ])
        ])
    ])
    
]

app.run_server(debug=False)


 * Running on http://127.0.0.1:18282/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2025 17:09:59] "GET /_alive_16e40798-e57c-498d-b26b-f7f04068e005 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:18282/


127.0.0.1 - - [20/Apr/2025 17:10:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 17:10:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 17:10:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 17:10:02] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2025 17:10:02] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2025 17:10:02] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [20/Apr/2025 17:10:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 17:10:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 17:10:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 17:10:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 17:10:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2025 17:10:04]